In [ ]:
#@title 🤗 AutoTrain
#@markdown In order to use this colab
#@markdown - Enter your [Hugging Face Write Token](https://huggingface.co/settings/tokens)
#@markdown - Enter your [ngrok auth token](https://dashboard.ngrok.com/get-started/your-authtoken)
huggingface_token = '' # @param {type:"string"}
ngrok_token = "" # @param {type:"string"}

#@markdown
#@markdown - Attach appropriate accelerator `Runtime > Change runtime type > Hardware accelerator`
#@markdown - click `Runtime > Run all`
#@markdown - Follow the link to access the UI
#@markdown - Training happens inside this Google Colab
#@markdown - report issues / feature requests [here](https://github.com/huggingface/autotrain-advanced/issues)

import os
os.environ["HF_TOKEN"] = str(huggingface_token)
os.environ["NGROK_AUTH_TOKEN"] = str(ngrok_token)
os.environ["AUTOTRAIN_LOCAL"] = "1"

!pip install -U autotrain-advanced > install_logs.txt 2>&1
!autotrain app --share

In [1]:
pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ju7ty0ig/unsloth_225bb76559b14f42bfac45d33c9aeb2e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ju7ty0ig/unsloth_225bb76559b14f42bfac45d33c9aeb2e
  Resolved https://github.com/unslothai/unsloth.git to commit 0eb61fbea728cdc8acd1f2fa1f6f71074f559ac0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.5 MB/s eta 0:

In [1]:
pip install --upgrade "transformers[torch]" git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-g19wl4fr
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-g19wl4fr
  Resolved https://github.com/huggingface/transformers.git to commit add43c4d09a89e3ad134efc8fd119350f363642e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.54.0.dev0-py3-none-any.whl size=11891096 sha256=828cf6b433ad52562b6307201c3c5d63845597cee4f82941790eac6d5ef737f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-g_djye41/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.1
    Uninstalling transformers-4.53.1:
      Successfully uninstalled transformers-4.53.1


In [2]:
from huggingface_hub import login
login()

In [5]:
import torch
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset # Import load_dataset

# Load the model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen3-0.6B", # This will work now
    max_seq_length = 2048,
    load_in_4bit = True,
)

# Configure for efficient training (PEFT/LoRA)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
)

# Load the dataset
train_dataset = load_dataset("json", data_files="/content/train_qwen3_converted.jsonl", split="train")

# Set up the trainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,  # Pass the loaded dataset object
    dataset_text_field = "text", # The column in your dataset with the text
    max_seq_length = 2048,
    args = TrainingArguments(
        output_dir = "outputs",
        num_train_epochs = 3,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 5,
    ),
)

# Start training
trainer.train()

==((====))==  Unsloth 2025.7.3: Fast Qwen3 patching. Transformers: 4.54.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Generating train split: 0 examples [00:00, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/1600 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,600 | Num Epochs = 3 | Total steps = 600
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 10,092,544 of 606,142,464 (1.67% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nickx (nickx-mass) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
5,4.832500
10,0.363300
15,0.039600
20,0.016000
25,0.011400
30,0.005200
35,0.003400
40,0.003400
45,0.001000
50,0.003000


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


TrainOutput(global_step=600, training_loss=0.044097873641100405, metrics={'train_runtime': 1186.9143, 'train_samples_per_second': 4.044, 'train_steps_per_second': 0.506, 'total_flos': 2438441533440000.0, 'train_loss': 0.044097873641100405})

In [6]:
# First, merge the LoRA layers into the base model
model = model.merge_and_unload()

# Choose a name for your new merged model repository
repo_name_merged = "clarksa/qwen3-0.6b-trains"

# Push the full merged model and tokenizer to the Hub
# This will take longer as you are uploading the entire model
model.push_to_hub(repo_name_merged, token=True)
tokenizer.push_to_hub(repo_name_merged, token=True)

print(f"Successfully uploaded the full merged model to: https://huggingface.co/{repo_name_merged}")

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


README.md:   0%|          | 0.00/590 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/576M [00:00<?, ?B/s]

Saved model to https://huggingface.co/clarksa/qwen3-0.6b-trains


README.md:   0%|          | 0.00/596 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Successfully uploaded the full merged model to: https://huggingface.co/clarksa/qwen3-0.6b-trains


In [17]:
import torch
from unsloth import FastLanguageModel
from transformers import pipeline

# 你的Hugging Face用户名和合并后的模型仓库名
# ⚠️ 替换成你自己的信息
merged_repo = "clarksa/qwen3-0.6b-trains"

# 1. 直接从你的仓库加载完整的模型
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = merged_repo, # ⬅️ 直接指向你的完整模型仓库
    load_in_4bit = False,
)

# 2. 准备推理
text_generator = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
)

# 3. 提问和生成
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "你接受了什么样的微调训练？"}, # 换成你想问的问题
]
prompt = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = True)

outputs = text_generator(prompt, max_new_tokens = 256)
print(outputs[0]['generated_text'])

==((====))==  Unsloth 2025.7.3: Fast Qwen3 patching. Transformers: 4.54.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ValueError: Supplied state dict for model.layers.0.self_attn.k_proj.weight does not contain `bitsandbytes__*` and possibly other `quantized_stats` components.

In [20]:
import torch
from unsloth import FastLanguageModel

# --- Configuration ---

# 1. 明确指定包含你的LoRA适配器的本地文件夹路径
#    Based on your screenshot, this is the correct path.
local_checkpoint_path = "outputs/checkpoint-600"

# 2. 为你的LoRA适配器创建一个新的、清晰的Hugging Face仓库名称
#    在末尾加上 "-lora" 是一个好习惯，以便区分。
#    ⚠️ 请将 "YourUsername/YourModelName" 替换为你的实际信息。
lora_repo_name = "clarksa/qwen3-0.6b-finetuned-lora"


# --- Execution ---

# 3. 从本地检查点文件夹加载模型和分词器
print(f"Loading model from local checkpoint: {local_checkpoint_path}...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = local_checkpoint_path,
    load_in_4bit = True,
)

# 4. 将加载的适配器和分词器推送到新的Hugging Face仓库
print(f"Pushing LoRA adapters to new repository: {lora_repo_name}...")
model.push_to_hub(lora_repo_name, token=True)
tokenizer.push_to_hub(lora_repo_name, token=True)

print("\n✅ Success!")
print(f"Your LoRA adapters are now saved to: https://huggingface.co/{lora_repo_name}")

Loading model from local checkpoint: outputs/checkpoint-600...
==((====))==  Unsloth 2025.7.3: Fast Qwen3 patching. Transformers: 4.54.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Pushing LoRA adapters to new repository: clarksa/qwen3-0.6b-finetuned-lora...
Saved model to https://huggingface.co/clarksa/qwen3-0.6b-finetuned-lora

✅ Success!
Your LoRA adapters are now saved to: https://huggingface.co/clarksa/qwen3-0.6b-finetuned-lora


In [ ]:
import torch
from unsloth import FastLanguageModel
from transformers import pipeline

# --- 模型加载部分 (和之前一样) ---
# 这部分只需要在程序开始时运行一次

print("正在加载模型，请稍候...")
lora_repo = "clarksa/qwen3-0.6b-finetuned-lora"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = lora_repo,
    load_in_4bit = True,
)

text_generator = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
)
print("\n✅ 模型加载完成！可以开始对话了。")
# --- 模型加载部分结束 ---


# --- 循环对话逻辑 ---

# 1. 初始化对话历史，并添加系统指令
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
]

while True:
    # 2. 获取用户输入
    user_input = input("你: ")

    # 3. 设置退出条件
    if user_input.lower() in ["exit", "quit", "退出"]:
        print("再见！")
        break

    # 4. 将用户输入添加到对话历史中
    messages.append({"role": "user", "content": user_input})

    # 5. 应用聊天模板
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # 6. 生成模型回复
    outputs = text_generator(
        prompt,
        max_new_tokens = 512,  # 可以适当增加长度以获得更完整的回复
        do_sample = True,
        temperature = 0.7,
        top_p = 0.9,
    )

    # 7. 从输出中提取模型的回复
    full_text = outputs[0]['generated_text']
    # 找到最后一个 "<|im_start|>assistant" 标记，并提取之后的内容
    assistant_response = full_text.split("<|im_start|>assistant\n")[-1].strip()

    print(f"模型: {assistant_response}")

    # 8. 将模型的回复也添加到对话历史中，以便模型记住它自己说过的话
    messages.append({"role": "assistant", "content": assistant_response})

正在加载模型，请稍候...
==((====))==  Unsloth 2025.7.3: Fast Qwen3 patching. Transformers: 4.54.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Device set to use cuda:0



✅ 模型加载完成！可以开始对话了。
你: 你好
模型: ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности
ности